# BERT fine tune DRCD

In [16]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
import json
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops, stops
import numpy as np
import random
import argparse
import time
from datetime import datetime, timedelta

In [17]:
import json

class DRCDRawData():
    def __init__(self, train_path=None , test_path=None ,dev_path=None):
        if train_path != None:
            self.train = self.load_data(train_path)
        if test_path != None:
            self.test = self.load_data(test_path)
        if dev_path != None:
            self.dev = self.load_data(dev_path)
    def load_data(self,path):
        dataset = json.load(open(path, encoding='utf-8'))
        result = []
        for i in range(len(dataset['data'])):
            for j in dataset['data'][i]['paragraphs']:
                context = j['context']
                for qa in j['qas']:
                    question = qa['question']
                    if qa['answers']:
                        result.append([context,question,qa['answers'][0]])                     
        return result

# Dataset

In [18]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class DRCD(Dataset):
    def __init__(self, data, model_type, device, language):
        self.data = data
        self.tokenizer = BertTokenizer.from_pretrained(model_type)
        self.device = device
        self.cc = OpenCC('t2s') # tw->china
        self.language = language
    
    def __getitem__(self, idx):
        
        paragraph, question, ans = self.data[idx][0], self.data[idx][1], self.data[idx][2] 
        
        if self.language == 'china':
            paragraph, question, ans = self.cc.convert(paragraph), self.cc.convert(question), self.cc.convert(ans)
        
        # Tokenize and prepare for the model a sequence or a pair of sequences. 
        # so unlike 'encode' just encode to ids, encode_plus return not only ids 
        # but token_type_ids, attention_mask also
        token_tensor = self.tokenizer.encode_plus(question, paragraph, max_length=512, 
                                                  truncation=True, pad_to_max_length=True)
        
        ans_encode = self.tokenizer.encode(ans)
        s_tensor, e_tensor = self.find_ans_index(token_tensor, ans_encode)

        return {'input_ids': torch.tensor(token_tensor['input_ids']).to(self.device),
                'token_type_ids': torch.tensor(token_tensor['token_type_ids']).to(self.device),
                'attention_mask': torch.tensor(token_tensor['attention_mask']).to(self.device),
                's_tensor': s_tensor.to(self.device),
                'e_tensor': e_tensor.to(self.device)}
    
    def __len__(self):
        return len(self.data)
    
    def find_ans_index(self, token_tensor, ans_encode):
        s_idx, e_idx = [0] * 512, [0] * 512

        # answer's token (removing [CLS] and [SEP] by [1:-1])
        s_tok = self.tokenizer.encode(ans)[1:-1]
        
        # find the idx of paragraph startwith ans's first token 
        s_list = [i for i, x in enumerate(token_tensor['input_ids']) if x == s_tok[0]]
      
        for s_pos in s_list:
            e_pos = s_pos + len(s_tok)
            if e_pos > 511:
                continue
            if token_tensor['input_ids'][s_pos:e_pos] == s_tok:
                s_idx[s_pos] = 1
                e_idx[e_pos-1] = 1
                break
        return  torch.Tensor(s_idx), torch.Tensor(e_idx)
    

# Model

In [28]:
class BertForReadingComprehension(nn.Module):
    def __init__(self, model_type):
        super(BertForReadingComprehension, self).__init__()

        config = BertConfig.from_pretrained(model_type, output_hidden_states=True)
        self.bert_model = BertModel.from_pretrained(model_type, config=config)
        
        self.s_decoder = nn.Sequential(nn.Linear(config.hidden_size, 1)) 
        self.e_decoder = nn.Sequential(nn.Linear(config.hidden_size, 1))

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None): 
        hidden = self.bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0]
        
        s = self.s_decoder(hidden).squeeze()
        e = self.e_decoder(hidden).squeeze()
        
        # mask question part, because ans only appear in paragraph.
        mask = token_type_ids.clone().float().to(hidden.device).detach()
        mask[mask != 1] = float('-inf')

        return s + mask, e + mask


# Test

In [20]:
def test(model, ds, args, tp):
    if tp == 'test':
        dataset = DRCD(ds.test, args.model_type, args.device, args.language)
        print(f'Testing...{len(dataset)}')
    elif tp == 'dev':   
        dataset = DRCD(ds.dev, args.model_type, args.device, args.language)
        print(f'Dev...{len(dataset)}')
    loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
    model.eval()
    loss,i = 0,0
    print(loss,i)
    criterion = nn.BCELoss()
    with torch.no_grad():
        for batch in loader:
            i = i+1
            inp_ids, tok_id, att_m, slabel, elabel = batch['input_ids'], batch['token_type_ids'], batch['attention_mask'], batch['s_tensor'], batch['e_tensor']
            s, e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
            #print(f's {s} \n e {e} \n sl {torch.softmax(s,dim=-1)} \n el {torch.softmax(e,dim=-1)} \n ')
            #print(f'SL {slabel} \n EL {elabel}')
            batch_loss = (criterion(torch.softmax(s,dim=-1),slabel) + criterion(torch.softmax(e,dim=-1),elabel)) / 2
            loss += batch_loss
            
        print(f'Result: LOSS: {loss} AVG: {loss/i} ')
    return loss

In [21]:
def train(ds,args):
    dataset = DRCD(ds.train, args.model_type, args.device, args.language)
    trainLoader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
    model = BertForReadingComprehension(args.model_type).to(args.device)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = AdamW(parameters, lr=args.learning_rate, weight_decay=args.weight_decay)
    criterion = nn.BCELoss()
    model.train()
    minloss = 100000
    for ei in range(args.epoch):
        model.train()
        epoch_loss,i,check_loss = 0,0,0
        for batch in trainLoader:
            i+=1
            inp_ids,tok_id,att_m ,slabel,elabel = batch['input_ids'],batch['token_type_ids'],batch['attention_mask'],batch['s_tensor'],batch['e_tensor']
            s,e = model(input_ids=inp_ids,attention_mask=att_m,token_type_ids=tok_id)
            #print(f's {s} \n e {e} \n sl {torch.softmax(s,dim=-1)} \n el {torch.softmax(e,dim=-1)} \n ')
            #print(f'SL {slabel} \n EL {elabel}')
            batch_loss = (criterion(torch.softmax(s,dim=-1),slabel) + criterion(torch.softmax(e,dim=-1),elabel)) / 2
            epoch_loss += batch_loss
            check_loss += batch_loss
            #print(batch_loss)
            batch_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if i % 1000==0:
                print(f'1000 batch LOSS {check_loss}')
                check_loss = 0
        dev_loss = test(model,ds,args, 'dev') 
        if dev_loss < minloss:
            minloss = dev_loss
            best_model = copy.deepcopy(model.state_dict())
        print(f'===Epoches: {ei} Loss {epoch_loss}===')       
    return best_model

# Main

In [22]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
parser = argparse.ArgumentParser([])
parser.add_argument('--batch-size', default=2 , type=int)
parser.add_argument('--epoch', default=5, type=int)
parser.add_argument('--learning-rate', default=1e-5, type=float)    
parser.add_argument('--weight-decay', default=0.001, type=float)
parser.add_argument('--model-type', default='hfl/chinese-roberta-wwm-ext' , type=str)  #model_type = 'hfl/chinese-bert-wwm'  'hfl/chinese-roberta-wwm-ext'  'hfl/chinese-roberta-wwm-ext-large'
parser.add_argument('--device', default=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'), type=int)
parser.add_argument('--language', default='tw', type=str)
args = parser.parse_args([])
args

True
Quadro P2000


Namespace(batch_size=2, device=device(type='cuda', index=0), epoch=5, language='tw', learning_rate=1e-05, model_type='hfl/chinese-roberta-wwm-ext', weight_decay=0.001)

In [24]:
data = DRCDRawData(train_path='./dataset/DRCD_train.json',
                   test_path='./dataset/DRCD_test.json',
                   dev_path='./dataset/DRCD_dev.json')
data.train[0]

['2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
 '廣州的快速公交運輸系統每多久就會有一輛巴士？',
 {'id': '1', 'text': '10秒鐘', 'answer_start': 84}]

In [25]:
mode = 'train'

if mode == 'train': 
    print('Train')
    best_model = train(data,args)
    if not os.path.exists('saved_models'):
        os.makedirs('saved_models')    
    model_name = 'BertForReadingComprehension'+'_'+(datetime.now()+timedelta(hours=8)).strftime("%m%d_%H%M")+'.pt' 
    torch.save(best_model, f'saved_models/{model_name}')
    print(f'Train end, model name is {model_name}')

elif mode == 'test' or mode == 'dev':
    model_name = 'bertDRCD_0808_1213.pt'
    test_model = BertForReadingComprehension(args.model_type).to(args.device)
    test_model.load_state_dict(torch.load(f'saved_models/{model_name}'))
    test(test_model, data, args, mode)

#22.53021812438965
#22.579822540283203
# Result: LOSS: 3.2912609577178955 AVG: 0.003765744622796774 
# Result: LOSS: 3.526034116744995 AVG: 0.004002308938652277 

Train


KeyboardInterrupt: 

# Package

In [29]:
class InferenceModel():
    def __init__(self, model_path, model_type, language): 
        # device
        # language define the language model wanna eat
        self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
        # self.device = torch.device('cpu')
        # model & tokenizer
        # model_type = 'hfl/chinese-roberta-wwm-ext'
        config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
        self.tokenizer = BertTokenizer.from_pretrained(model_type)
        self.model = BertForReadingComprehension(model_type).to(self.device)
        self.model.load_state_dict(torch.load(model_path)) 
        self.language = language

        self.c2tw = OpenCC('s2t') # china to tw
        self.tw2c = OpenCC('t2s') # tw to china

    def inference(self,content,question):
        
        content, question = self.clean_str(content, question)
        
        
        with torch.no_grad():
            
            token_tensor = self.tokenizer.encode_plus(str(question), str(content), max_length=512, 
                                                      truncation=True, pad_to_max_length=True)
            token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
            segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
            mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
            answer_start, answer_end = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment) 
      
            # get ans
            tokens = self.tokenizer.convert_ids_to_tokens(token.squeeze())
            answer_start = answer_start.argmax(1)
            answer_end = answer_end.argmax(1)
            #print(answer_start,answer_end)
            if answer_start > answer_end :
                return '抱歉 這題有點難耶'
            
            #print(answer_start,answer_end)
            answer = ''.join(tokens[answer_start:answer_end+1])
            if self.language == 'tw':
                answer = self.c2tw.convert(answer)
            
            
        
        return answer
    
    def clean_str(self,content,question):
        content = content.replace(' ','')
        question = question.replace(' ','')
        if self.language == 'china':
            content = self.tw2c.convert(content)
            question = self.tw2c.convert(question)

        return content,question

In [30]:
print(torch.__version__)
model_name = 'bertDRCD_1023_1619.pkl'
drcd_model = InferenceModel(f'saved_models/{model_name}','hfl/chinese-roberta-wwm-ext','tw')

1.3.1


In [32]:
content = \
'蒂埃里·亨利是前法國足球運動員。在他職業生涯參加的國際賽事中，他為法國隊出場123次，\
打進51球。他的首個國際比賽進球是在1998年世界盃對陣南非的比賽中。截至2015年10月，他是法國隊的頭號射手\
。2007年10月，他在對陣立陶宛的比賽中打進兩球，打破了米歇爾·普拉蒂尼42球的法國隊進球紀錄。亨利於2010年7月正式退役\
。亨利在2009年10月對奧地利的比賽中打進了個人國家隊第51粒進球，這也是他的最後一粒進球\
。亨利從未在國際比賽中上演過帽子戲法，儘管他曾7次在單場比賽梅開二度。\
他在對陣馬爾他的比賽中進球最多，在2004年歐洲國家盃外圍賽中曾單場上演大四喜。\
亨利一半以上的進球來自於主場比賽，他的51個進球中有31個是在法國本土打進，\
其中有20個在法蘭西體育場。亨利在熱身賽中共打進16球。在2003年國際足總洲際國家盃上\
，亨利打進四球並榮膺最佳射手，他也因此被評為「賽事最傑出球員」。\
亨利在歐洲國家盃外圍賽中打入12球，其中在2004年歐洲杯外圍賽打進6球，他最終排在射手榜第三位。'

question = ['蒂埃里·亨利是誰','蒂埃里·亨利為法國隊出場幾次','亨利曾在國際比賽中上演過帽子戲法嗎?',
            '蒂埃里·亨利是誰在哪年歐洲國家盃外圍賽中曾單場上演大四喜?','亨利在熱身賽中共打進多少球?']

for q in question:
    print(q + ':  '+ drcd_model.inference(content,q))

蒂埃里·亨利是誰:  前法國足球運動員
蒂埃里·亨利為法國隊出場幾次:  123次
亨利曾在國際比賽中上演過帽子戲法嗎?:  從未
蒂埃里·亨利是誰在哪年歐洲國家盃外圍賽中曾單場上演大四喜?:  2004年
亨利在熱身賽中共打進多少球?:  16球


In [34]:
content = '2020年4月17日，高雄市選舉委員會公告罷免有效聯署書達到37.7萬，罷韓第二階段通過\
，韓國瑜成為全國第一位被罷免投票兩次者、也成為首個市長罷免案進入投票階段的市長。2020年6月6日\
，罷免投票通過，6月12日中央選舉委員會公告結果後正式解職\
，韓國瑜成為中華民國選舉史上首位被罷免成功的縣市首長。\
韓國瑜曾自述，他在年少求學階段考試都考第一名，但國中二年級開始因青春期懵懂，\
每天上課一直看前排女同學的小腿，成績一落千丈，國中三年級被轉入放牛班，逃學翹課\
、打撞球、打架也樣樣俱全，最後在父母決定下，18歲時入中華民國陸軍軍官學校專修學生班40期\
，軍銜升至上尉，自稱在軍中受一位國立臺灣大學醫學系畢業的預官轉魔術方塊的啟發，感到自己的不足\
，決定重拾課本。退伍前一年考上東吳大學英國語文學系，\
畢業後又考取國立政治大學東亞研究所和淡江大學國際事務與戰略研究所。\
由於政大免學費還發獎學金故選擇政大東亞所就讀。就讀期間擔任臺北市議員馮定國的助理，\
畢業後獲法學碩士學位。碩士論文名稱為《從中共「對臺統戰」策略看兩航談判》，指導教授為蘇起。韓國瑜喜歡'

question = ['罷免韓國瑜第二階段有通過嗎','中華民國選舉史上首位被罷免成功的縣市首長是誰'
            ,'罷免有效聯署書達到多少?','韓國瑜在哪天正式解職','韓國瑜幾歲被轉入放牛班'
            ,'韓國瑜每天上課一直看誰小腿','韓國瑜的碩士論文名稱?',
            '韓國瑜退伍前一年考上什麼','韓國瑜為什麼成績一落千丈','韓國瑜擔任誰的助理', '韓國瑜喜歡什麼?']

for q in question:
    print(q + ':  '+ drcd_model.inference(content,q))

罷免韓國瑜第二階段有通過嗎:  2020年
中華民國選舉史上首位被罷免成功的縣市首長是誰:  韓國瑜
罷免有效聯署書達到多少?:  37.7萬
韓國瑜在哪天正式解職:  6月12日
韓國瑜幾歲被轉入放牛班:  國中三年級
韓國瑜每天上課一直看誰小腿:  前排女同學
韓國瑜的碩士論文名稱?:  《從中共「對臺統戰」策略看兩航談判》
韓國瑜退伍前一年考上什麼:  東吳大學英國語文學系
韓國瑜為什麼成績一落千丈:  青春期懵懂
韓國瑜擔任誰的助理:  馮定國
韓國瑜喜歡什麼?:  兩航談判


In [35]:
content = '賈伯斯在1970年代末與蘋果公司另一始創人史蒂夫·沃茲尼克及首任投資者邁克·馬庫拉協同其他人設計、\
開發及銷售Apple II系列。在1980年代初，賈伯斯是最早看到全錄帕洛奧圖中心（Xerox PARC）的滑鼠驅動圖形用戶介面的商業潛力，\
並將其應用於Apple Lisa及一年後的麥金塔電腦。1985年，在董事會的鬥爭失勢後，賈伯斯離開蘋果公司及成立了NeXT公司(一間電腦\
平台開發公司，專門從事高等教育及商業市場)在1986年，他收購了盧卡斯影業的電腦繪圖部門，成立了皮克斯（Pixar）。\
他被譽為《玩具總動員》（1995年）的執行製片人。他一直擔任皮克斯動畫的執行長並持有50.1%的股份，\
直到公司在2006年被華特迪士尼公司收購，此項收購使賈伯斯成為迪士尼公司的最大個人股東（有7.4%的股份）及董事會成員。\
在1996年，蘋果公司董事會決議買下NeXT公司，把賈伯斯帶回他參與創立，卻正在垂死邊緣的蘋果公司擔任臨時CEO。\
他在2000年起成為正式CEO，帶領蘋果輝煌的iPod、iPhone、iPad時代的到來。從2003年10月起，賈伯斯與胰腺神經內分泌腫瘤奮戰了8年，\
最終於2011年8月辭任執行長一職，在他第3次病假期間，賈伯斯當選為蘋果公司的董事長。'


question = ['NeXT公司是誰創造的','NeXT公司是在哪年成立','誰決議買下NeXT公司','誰成立了皮克斯'
            ,'誰是《玩具總動員》的執行製片人','皮克斯動畫在哪年被華特迪士尼公司收購']


for q in question:
    print(q + ':  '+ drcd_model.inference(content,q))



NeXT公司是誰創造的:  賈伯斯
NeXT公司是在哪年成立:  1985年
誰決議買下NeXT公司:  蘋果公司另一始創人史蒂夫·沃茲尼克及首任投資者邁克·馬庫拉協同其他人設計、開發及銷售apple##ii系列。在1980年代初，賈伯斯是最早看到全錄帕洛奧圖中心（x##er##ox##pa##rc）的滑鼠驅動圖形用戶介面的商業潛力，並將其應用於apple##lis##a及一年後的麥金塔電腦。1985年，在董事會
誰成立了皮克斯:  賈伯斯
誰是《玩具總動員》的執行製片人:  賈伯斯
皮克斯動畫在哪年被華特迪士尼公司收購:  2006年
